# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Random Word generator

In [5]:
def word_generator(length=5)
  rstr = ''
  length.times{|k| rstr += Rseq.sample.to_s}
  return Word.new(rstr)
end
def wordgen(length=5)
  word_generator(length)
end;nil

In [6]:
p wordgen(10).show

"DEfcCFCdFa"


"DEfcCFCdFa"

# Goldman-Turaev bialgebra

## Linking of a pair of divisions

In [7]:
def ep(h1, h2)
  result = h1[:sign]*h2[:sign]
  result *= if h1[:letter].show == h2[:letter].show
    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  else
    (Rseq.index(h1[:letter]) < Rseq.index(h2[:letter])) ? 1 : -1
  end
  return result
end
#--------------------------------------------

def lk(h1, h2) # h1={word: w1, div: div1}, h2={word: w2, div: div2})
  div_hashs = [h1,h2].map do |h|
    w = h[:word].flatten
    k = h[:div]
    s = w.size
    [{index: (k%s)+1, letter: w[k%s], sign: 1}, {index: k, letter: w[k-1].inverse, sign: -1}]
  end
  sum = 0
  2.times do |i|
    2.times{|j| sum += ep(div_hashs[0][i], div_hashs[1][j])}
  end
  return (-1)*(sum/2)
end; nil

## Expansion Class

In [342]:
class Expansion < Array
  def show(simplify=false)
    expn = simplify ? self.simplify : self
    mstr = expn.map do |h|
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      words = h[:words]
      term = (words.size == 1) ? words[0].show : words.map(&:cyclic_reduce).join("\u{2227}")
      coeff + term
    end.join(' + ')
    return mstr
  end

  def simplify()
    #-----
    expn = self.class.new
    monomials = self.each_with_object([]) do |mono, arr|
      if mono[:coeff] != 0
        mono[:words].map!(&:cyclic_reduce)
        arr << mono
      end
    end
    while monomials.size > 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        flag = false
        (mono[:words].size).times do |i|
          mono[:words].zip(pop[:words].rotate(i)).each do |couple|
            flag = true if couple[0].conjugate?(couple[1])
          end      
        end
        if flag
          pop[:coeff] += mono[:coeff]
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      #---
      expn << pop if pop[:coeff] != 0
      #---
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    #-----
    if expn.empty?
      return (self.class.new << {divs: [], coeff: 0, words: [Word.new]})
    else
      return expn
    end
  end
end; nil

In [9]:
expn = Expansion.new() 
5.times do |i|
#  expn << {divs: '', coeff: (-1)**i, words: [Word.new]}
   expn << {divs: '', coeff: (-1)**i, words: [Word.new, Word.new]}
end
p expn.show, expn.simplify.show
nil

"1∧1 + (-1)1∧1 + 1∧1 + (-1)1∧1 + 1∧1"
"1∧1"


## Goldman bracket

In [10]:
def bracket(w1, w2)
  """
  retern an Expansion (an Array of Hashes {divs: *, coeff: *, words: *})
  """
  #-----
  polynomial = Expansion.new
  if w1.show != '1' and w2.show != '1' 
    w1.size.times do |i|
      #--- divide two words each ---
      div1 = i+1
      nw1 = w1.cyclic_permutation(div1)
      w2.size.times do |j|
        div2 = j+1
        #--- linking of the pair of divisions ---
        lk = lk({word: w1, div: div1}, {word: w2, div: div2})
        nw2 = w2.cyclic_permutation(div2)
        #---
        polynomial << {divs: [div1, div2], coeff: lk, words: [nw1*nw2]}
      end
    end
  end
  return polynomial
end; nil

In [22]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
w1, w2 = wordgen(2), wordgen(3)
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

# bra.each do |h|
#   h.each{|k,v| p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)}
#   printf "---\n"
# end

p bra.show
p bra.simplify.show
; nil

AB, eaB
-------
"BA.aBe + (0)BA.Bea + (0)BA.eaB + (0)AB.aBe + (0)AB.Bea + AB.eaB"
"ABeaB + BBe"


## Turaev cobraket

In [23]:
def cobracket(myw, verbose: false)
  """
  return an Expansion (an Array of Hashes {divs: *, coeff: *, words: *})
  """
  polynomial = Expansion.new
  unless myw.show == '1' 
    divs_combinations = (1..myw.size).to_a.combination(2)  #2つの div の組み合わせの全体
    divs_combinations.each do |combi|
      #--- linking of the pair of divisions ---
      lk = lk({word: myw, div: combi[0]},{word: myw, div: combi[1]})
      #--- divide the word --- 
      facs = myw.flatten.factors
      words = Array.new(2)
      #--- [[TODO]] 以下の2つの Word の定義に cyclic_permutation を使えないか?
      words[0] = Word.new(facs[0..combi[0]-1]+facs[combi[1]..-1])
      words[1] = Word.new(facs[combi[0]..combi[1]-1])    
      polynomial << {divs: combi, coeff: lk, words: words}
    end
  end
  #-----
  return polynomial
end; nil

In [53]:
#mw = Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
mw = wordgen(5)
printf mw.show + "\n-------\n"

cobra = cobracket(mw, verbose: false) #true) #

# cobra.each do |h|
#   h.each do |k,v| 
#     p "#{k}: " + ((k == :words) ? v[0].show : v.to_s)
#   end
#   printf "---\n"
# end

p cobra.show
p cobra.simplify.show
nil

BFEcD
-------
"(0)BEcD∧F + (0)BcD∧FE + (0)BD∧FEc + (-1)B∧FEcD + (0)BFcD∧E + (0)BFD∧Ec + (0)BF∧EcD + (0)BFED∧c + BFE∧cD + (0)BFEc∧D"
"BFE∧cD + (-1)B∧FEcD"


## Execution

In [ ]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s, verbose: false).show }
end;nil

# Conditions

## Jacobi Identity

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [749]:
three_words = Array.new(3)
max_length = 3

three_words.map!{|w| word_generator(rand(1..max_length))}
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> \n\n------"

total = Expansion.new
3.times do |k|
  bra = bracket(three_words[0],three_words[1])
  puts "\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
  bra.each do |h|
    bra2 = bracket(h[:words][0], three_words[2])
    puts "+ (#{bra2.show}) \u{2297} #{three_words[2].show}"
    bra2.each do |hh|
      hh[:coeff] *= h[:coeff]
      total << hh
    end
  end
  three_words.rotate!
end

printf "---------\n"
p total.show
printf "---------\n"
p total.simplify.show
nil

D ⊗ EDc ⊗ AbD  |---> 

------
∇(D ⊗ EDc) ⊗ AbD =
+ ((0)DcED.bDA + (-1)DcED.DAb + DcED.AbD + (0)cEDD.bDA + (-1)cEDD.DAb + cEDD.AbD + (0)EDDc.bDA + (0)EDDc.DAb + (-1)EDDc.AbD + (0)DDcE.bDA + (-1)DDcE.DAb + DDcE.AbD) ⊗ AbD
+ ((0)cEDD.bDA + (-1)cEDD.DAb + cEDD.AbD + (0)EDDc.bDA + (0)EDDc.DAb + (-1)EDDc.AbD + (0)DDcE.bDA + (0)DDcE.DAb + DDcE.AbD + (0)DcED.bDA + (-1)DcED.DAb + (0)DcED.AbD) ⊗ AbD
+ ((0)EDcD.bDA + (0)EDcD.DAb + (0)EDcD.AbD + (0)DcDE.bDA + (0)DcDE.DAb + DcDE.AbD + (0)cDED.bDA + (-1)cDED.DAb + (0)cDED.AbD + (0)DEDc.bDA + (-1)DEDc.DAb + (0)DEDc.AbD) ⊗ AbD
∇(EDc ⊗ AbD) ⊗ D =
+ ((-1)cEbDAD.D + (-1)EbDADc.D + (0)bDADcE.D + DADcEb.D + (-1)ADcEbD.D + DcEbDA.D) ⊗ D
+ ((-1)cEDAbD.D + (-1)EDAbDc.D + DAbDcE.D + (-1)AbDcED.D + (0)bDcEDA.D + DcEDAb.D) ⊗ D
+ ((-1)cEAbDD.D + (-1)EAbDDc.D + (0)AbDDcE.D + (0)bDDcEA.D + DDcEAb.D + (0)DcEAbD.D) ⊗ D
+ ((-1)EDbDAc.D + DbDAcE.D + (-1)bDAcED.D + DAcEDb.D + (-1)AcEDbD.D + (0)cEDbDA.D) ⊗ D
+ ((-1)EDDAbc.D + DDAbcE.D + (0)DAbcED.D + (-1)AbcEDD.D + (0)bc

In [748]:
three_words = Array.new(3)
max_length = 3
count = 500

count.times do |k|
  three_words.map!{|w| word_generator(rand(1..max_length))}
  total = Expansion.new
  3.times do |k|
    bra = bracket(three_words[0],three_words[1])
#     puts "\u{2207}(" + three_words[0..1].map(&:show).join(" \u{2297} ") + ") \u{2297} " + three_words[2].show + " ="
    bra.each do |h|
      bra2 = bracket(h[:words][0], three_words[2])
#       puts "+ (#{bra2.show}) \u{2297} #{three_words[2].show}"
      bra2.each do |hh|
        hh[:coeff] *= h[:coeff]
        total << hh
      end
    end
    three_words.rotate!
  end
  result = total.show(1)
  puts three_words.map(&:show).join(" \u{2297} ") + "  |--> " + result if result != '(0)1'
end; nil

Ada ⊗ FFD ⊗ aA  |--> FF + (-1)FF + (-1)FF + FF
FE ⊗ E ⊗ fE  |--> (-1)EEE + EEE
c ⊗ D ⊗ DC  |--> (-1)DD + DD


## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [745]:
myw = word_generator(5)
#myw = Word.new('ffA')
puts myw.show + "\n======"

cobra = cobracket(myw)
puts "\u{0394}(#{myw.show}) = " + cobra.show
puts "---"

total = Expansion.new
cobra.each do |h|
  coeff = h[:coeff]
  bra = bracket(*h[:words])
  bra.each do |mono|
    mono[:coeff] *= coeff
    total << mono
  end
  #--- display ---
  tensor = h[:words].map(&:show).join(" \u{2297} ")
  puts "\t\u{2207}(#{tensor}) = #{bra.show}"
  #---------------
end

puts "======\n" + total.show
puts "======\n" + total.show(1)
nil

AEfeA
Δ(AEfeA) = (-1)AfeA∧E + (0)AeA∧Ef + (0)AA∧f + A∧EfeA + (0)AA∧f + (-1)AEA∧fe + (0)AE∧feA + (0)AEfA∧e + (0)AEf∧eA + (0)AEfe∧A
---
	∇(AfeA ⊗ E) = (0)feAA.E + (0)eAAf.E + (-1)AAfe.E + (0)AfeA.E
	∇(AeA ⊗ Ef) = (0)eAA.fE + (0)eAA.Ef + (0)AAe.fE + (0)AAe.Ef + (0)AeA.fE + (0)AeA.Ef
	∇(AA ⊗ Efe) = (0)AA.feE + (0)AA.eEf + (0)AA.Efe + (0)AA.feE + (0)AA.eEf + (0)AA.Efe
	∇(A ⊗ EfeA) = (0)A.feAE + (0)A.eAEf + (0)A.AEfe + (0)A.EfeA
	∇(AEeA ⊗ f) = (0)EeAA.f + (0)eAAE.f + (0)AAEe.f + (0)AEeA.f
	∇(AEA ⊗ fe) = (0)EAA.ef + EAA.fe + (0)AAE.ef + (0)AAE.fe + (0)AEA.ef + (0)AEA.fe
	∇(AE ⊗ feA) = (0)EA.eAf + (0)EA.Afe + (0)EA.feA + (0)AE.eAf + (0)AE.Afe + (0)AE.feA
	∇(AEfA ⊗ e) = (0)EfAA.e + (0)fAAE.e + (0)AAEf.e + (0)AEfA.e
	∇(AEf ⊗ eA) = (0)EfA.Ae + (0)EfA.eA + (0)fAE.Ae + (0)fAE.eA + (0)AEf.Ae + (0)AEf.eA
	∇(AEfe ⊗ A) = (0)EfeA.A + (0)feAE.A + (0)eAEf.A + (0)AEfe.A
(0)feAA.E + (0)eAAf.E + (-1)AAfe.E + (0)AfeA.E + (0)eAA.fE + (0)eAA.Ef + (0)AAe.fE + (0)AAe.Ef + (0)AeA.fE + (0)AeA.Ef + (0)AA.feE + (0)AA

In [746]:
count, length = 1000, 5
count.times do |k|
  myw = word_generator(length)
  total = cobracket(myw).each_with_object(Expansion.new) do |h, expn|
    bracket(*h[:words]).each do |mono|
      mono[:coeff] *= h[:coeff]
      expn << mono
    end
  end
  result = total.show(1)
  p myw.show + " |--> " + result if result != "(0)1"
end; nil

"ffEef |--> fff + (-1)fff + fff + (-1)fff + fff + (-1)fff"
"FDFdF |--> (-1)FFF + FFF + FFF + (-1)FFF + FFF + (-1)FFF + FFF + (-1)FFF"
"BBBEe |--> BBB + (-1)BBB + BBB + (-1)BBB"
"CCdDC |--> CCC + (-1)CCC"


## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = w\cdot\delta(v)- v\cdot\delta(w),
$$
where 
$$
w\cdot(x\otimes y) = [w,x]\otimes y + x\otimes[w,y].
$$

In [702]:
v = word_generator(3) #samples[:s2][0] #Word.new(a[1], a[2])
w = word_generator(3) #samples[:s4][0] #Word.new(b[1])
p (v*w).show; nil

"fac.dfc"


In [703]:
lhe = Expansion.new
bracket(v, w).each do |h|
  cobracket(h[:words][0]).each do |mono|
    mono[:coeff] *= h[:coeff]
    lhe << mono
  end
end

printf "Left-hand Equation: \n"+ lhe.show(1)
nil

Left-hand Equation: 
(-2)facdf∧c + (-4)f∧accdf + (-2)ff∧accd + (2)cfa∧cdf + (2)cff∧acd

In [704]:
rhe = [] #Expansion.new
2.times do |i|
  pair = [v,w].rotate(i)
  coeff = (-1)**i
  #---
  cobra = cobracket(pair[1])
  printf "#{pair[0]}.\u{03B4}(#{pair[1].show}) = #{pair[0]}.(#{cobra.show})\n---\n"
  cobra.each do |h|
    if h[:coeff] != 0
      printf "\t[#{pair[0]}, #{h[:words][0]}] \u{2297} #{h[:words][1]} \n"
      bra = bracket(pair[0], h[:words][0])
      printf "\t=(#{bra.show}) \u{2297} #{h[:words][1]}"
      bra.each do |mono|
        if mono[:coeff] != 0
          mono[:coeff] *= coeff*h[:coeff]
          term = "(#{mono[:coeff]})#{mono[:words][0].show} \u{2297} #{h[:words][1]}\n"
          printf "\t=" + term
          rhe << term
        end
      end
      printf "\n\t#{h[:words][0]} \u{2297} [#{pair[0]}, #{h[:words][1]}] \n"
      bra = bracket(pair[0], h[:words][1])
      printf "\t=#{h[:words][0]} \u{2297} (#{bra.show}) \n"
      bra.each do |mono|
        if mono[:coeff] != 0
          mono[:coeff] *= coeff*h[:coeff]
          term = "(#{mono[:coeff]})#{h[:words][0]} \u{2297} #{mono[:words][0].show}\n"
          printf "\t=" + term
          rhe << term
        end
      end
    end
  end
  printf "---\n"
end
printf rhe.join(' + '); nil

fac.δ(dfc) = fac.((-1)dc∧f + (0)d∧fc + (0)df∧c)
---
	[fac, dc] ⊗ f 
	=((0)acf.cd + (0)acf.dc + cfa.cd + (0)cfa.dc + (0)fac.cd + (0)fac.dc) ⊗ f	=(-1)cfa.cd ⊗ f

	dc ⊗ [fac, f] 
	=dc ⊗ ((0)acf.f + (0)cfa.f + (0)fac.f) 
---
dfc.δ(fac) = dfc.((0)fc∧a + f∧ac + (0)fa∧c)
---
	[dfc, f] ⊗ ac 
	=((0)fcd.f + (0)cdf.f + (0)dfc.f) ⊗ ac
	f ⊗ [dfc, ac] 
	=f ⊗ ((0)fcd.ca + (-1)fcd.ac + (0)cdf.ca + cdf.ac + (0)dfc.ca + (-1)dfc.ac) 
	=(1)f ⊗ fcd.ac
	=(-1)f ⊗ cdf.ac
	=(1)f ⊗ dfc.ac
---
(-1)cfa.cd ⊗ f
 + (1)f ⊗ fcd.ac
 + (-1)f ⊗ cdf.ac
 + (1)f ⊗ dfc.ac


# Experiments

# TODO

* [ ] Compatibility がどこかおかしい。
* [ ] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある division について。